In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
gdf = gpd.read_file('/Terry/georeference/sls_5206_crs_3857.geojson')

In [3]:
df = pd.DataFrame(columns=['idsls', 'xmin', 'xmax', 'ymin', 'ymax'])

In [4]:
# Calculate the expansion percentage
expand_percentage = 0.025
target_ratio_landscape = (324, 272)
target_ratio_portrait = (278, 315)

# Iterate over each polygon
for index, row in gdf.iterrows():
    polygon = row['geometry']
    xmin, ymin, xmax, ymax = polygon.bounds
    
    width = xmax - xmin
    height = ymax - ymin
    
    # Calculate the expansion amount
    expand_amount = max(width, height) * expand_percentage
    
    if width > height:
        xmin = xmin - expand_amount
        xmax = xmax + expand_amount
        
        width = xmax - xmin
        height = ymax - ymin
        
        # Landscape orientation, adjust to target_ratio_landscape
        new_height = width * (target_ratio_landscape[1] 
                              / target_ratio_landscape[0])
        delta = (new_height - height) / 2

        ymin -= delta
        ymax += delta
        
    else:
        ymin = ymin - expand_amount
        ymax = ymax + expand_amount
        
        width = xmax - xmin
        height = ymax - ymin
        
        # Portrait orientation, adjust to target_ratio_portrait
        new_width = height * (target_ratio_portrait[0] 
                              / target_ratio_portrait[1])
        delta = (new_width - width) / 2

        xmin -= delta
        xmax += delta
    
    temp = pd.DataFrame({
      'idsls': row['idsls'],
      'xmin': xmin,
      'xmax': xmax,
      'ymin': ymin,
      'ymax': ymax
    }, index=[0])
    
    df = pd.concat([df, temp], ignore_index=True)

C:\Users\ipds5\AppData\Local\Temp\ipykernel_6740\2292685312.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp], ignore_index=True)


In [5]:
df.to_csv('/Terry/georeference/sls_st_coordinates.csv', index=False)